# Fine-tuning BERT with different optimizers (non-clipped and clipped)

### Dataset

[Yelp Reviews](https://huggingface.co/datasets/yelp_review_full) from HuggingFace

Data Fields:
- `text`: The review texts are escaped using double quotes ("), and any internal double quote is escaped by 2 double quotes (""). New lines are escaped by a backslash followed with an "n" character, that is "\n".
- `label`: Corresponds to the score associated with the review (between 1 and 5).


### Optimizers

- AdamW
- Nesterov-SGD
- clipped-Nesterov-SGD
- clipped-SSTM (~Nesterov-SGD with momentum)

### Training Code

Taken from HuggingFace documentation, page [Fine-tune a pretrained model](https://huggingface.co/docs/transformers/training#train-in-native-pytorch).

In [25]:
import torch
import wandb

In [2]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to C:/Users/newro/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [8]:
# tokenized_datasets = tokenized_datasets.remove_columns(["text"])
# tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [33]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_val_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(300))

Loading cached shuffled indices for dataset at C:\Users\newro\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-ba07a7a7a7e12e12.arrow
Loading cached shuffled indices for dataset at C:\Users\newro\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-cf36035e2430a8ac.arrow


In [34]:
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(300, 800))

Loading cached shuffled indices for dataset at C:\Users\newro\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-cf36035e2430a8ac.arrow


In [35]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
val_dataloader = DataLoader(small_val_dataset, shuffle=True, batch_size=8)

In [36]:
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [46]:
del model
torch.cuda.empty_cache()

In [47]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
from torch.optim import AdamW
from torch.optim import SGD

# optimizer = AdamW(model.parameters(), lr=5e-5)
optimizer = SGD(model.parameters(), lr=5e-5)

In [49]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [50]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [51]:
wandb.init(entity="makharev", project="iu-omml")

In [52]:
from tqdm.auto import tqdm
from torch.nn.utils import clip_grad_norm_

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        # Add gradient clipping
        clip_grad_norm_(model.parameters(), max_norm=1.0)  # L2-norm by default

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

        # Log loss to W&B
        wandb.log({"Training Loss": loss.item()})

        # Validation step
        if progress_bar.n % len(train_dataloader) == 0:
            model.eval()
            metric = evaluate.load("accuracy")
            for val_batch in val_dataloader:
                val_batch = {k: v.to(device) for k, v in val_batch.items()}
                with torch.no_grad():
                    val_outputs = model(**val_batch)

                val_predictions = torch.argmax(val_outputs.logits, dim=-1)
                metric.add_batch(predictions=val_predictions, references=val_batch["labels"])

            wandb.log({"Validation Accuracy": metric.compute()})
            model.train()

  0%|          | 0/375 [00:00<?, ?it/s]

In [53]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)    

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])


wandb.log({"Evaluation Accuracy": metric.compute()})
wandb.finish()